![](https://raw.githubusercontent.com/DivLoic/mdd-structured-streaming/master/resources/intro.png)

### Spark Structured Streaming

![](https://raw.githubusercontent.com/DivLoic/mdd-structured-streaming/master/resources/unbounded.png)

[Développer une application de Machine Learning en moins de 30 min - Alban Phelip & Mouloud Lounaci](https://youtu.be/iZoVwBDYyMU)
![](https://raw.githubusercontent.com/DivLoic/mdd-structured-streaming/master/resources/twitter.png)

#### Les sources :

![](https://raw.githubusercontent.com/DivLoic/mdd-structured-streaming/master/resources/source.png)

configuration

In [ ]:
val BROKER_HOST = "172.16.41.136"
val BROKER_PORT = "9092"
val S3_DIR = "/mnt/moisdeladata/"
val TOPIC = "twitter"

In [ ]:
println("***" * 30)
println("*")
println(s"* \t BROKER_HOST: $BROKER_HOST")
println(s"* \t BROKER_PORT: $BROKER_PORT")
println(s"* \t S3_DIR NAME: $S3_DIR")
println(s"* \t TOPIC NAME : $TOPIC")
println("*")
println("***" * 30)

In [ ]:
import org.apache.spark.sql.DataFrame

In [ ]:
/**
 * step 1: read from the source
 */

val df: DataFrame = spark.readStream.format("kafka")
    
    .option("kafka.bootstrap.servers", s"$BROKER_HOST:$BROKER_PORT")

    .option("subscribe", TOPIC) // list or regex
  
    .load()

In [ ]:
df.getClass

In [ ]:
df.isStreaming

In [ ]:
df.printSchema

#### Les transformations :

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [ ]:
println("get_json_object(col: Column, path: String): Column -- \"json string\"")
println()
println("from_json(col: Column, schema: StructType): Column  -- \"struct type\"")
println()
println("to_json(col: Column): Column  -- \"json string\"") 

kafka-connect-twitter : `{"schema": {}, "payload": {}}`

In [ ]:
/**
 * step 2: extract the payload
 */

val dfPayload = df.withColumn(
    "payload",
    get_json_object(
        $"value".cast(StringType), 
        "$.payload"
    )
)

In [ ]:
dfPayload.printSchema

```| <topic> | <timestamp> | {"id": 12345, "user": "info", ...} |```

In [ ]:
/**
 * step 3: build a apply a schema
 */

val ex_schema = new StructType(
    Array(
        StructField("id", StringType),
        StructField("media", StringType),
        StructField("text", StringType, false)
    )
)

In [ ]:
dfPayload.select(
    
    $"topic",
    $"timestamp",
    from_json($"payload", ex_schema) as 'tweet
    
).printSchema

In [ ]:
dfStructured.printSchema

```
root
 |-- topic: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- tweet: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- text: string (nullable = false)
 |    |-- created_at: string (nullable = true)
 |    |-- is_retweet: boolean (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: string (containsNull = false)
 |    |-- user: struct (nullable = true)
 |    |    |-- id: long (nullable = false)
 |    |    |-- location: string (nullable = true)
 |    |    |-- verified: boolean (nullable = true)
 |    |    |-- screen_name: string (nullable = true)
 |    |-- entities: struct (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = false)
 |    |    |    |    |-- text: string (nullable = true)
```

In [ ]:
/**
 * step 4: flatten the frame
 */

val dfTweets: DataFrame = dfStructured.select(
    $"topic",
    $"timestamp",
    $"tweet.id" as "tweet_id",
    $"tweet.text" as "text",
    $"tweet.is_retweet" as "is_retweet",
    substring($"tweet.created_at", 0, 23) as "created_at",
    unix_timestamp(
      substring($"tweet.created_at", 0, 23), 
      "yyyy-MM-dd'T'HH:mm:ss.S"
    ).cast(TimestampType) as "creation_time",
    $"tweet.user.id" as "user_id",
    $"tweet.user.verified" as "verified",
    $"tweet.user.location" as "location",
    $"tweet.user.screen_name" as "screen",
    $"tweet.entities.hashtags.text" as "hashtags"
 )

In [ ]:
dfTweets.printSchema

```
root
 |-- topic: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- is_retweet: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- creation_time: timestamp (nullable = true)
 |-- user_id: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- screen: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = false)
```

In [ ]:
/**
 * step 5: filter, join & display
 */

In [ ]:
dfTweets.createOrReplaceTempView("all_tweets")

In [ ]:
dfTweets

.filter(size($"hashtags") >= 4)

.createOrReplaceTempView("all_tweets")

In [ ]:
val ref = Seq(
  
  ("25073877", "@realDonaldTrump", "target1"),
  ("52544275", "@IvankaTrump", "target2"),
  ("822215679726100480", "@POTUS", "target3"),
  ("22203756", "@mike_pence", "target4"),
  ("<your-id>", "@<your-name>", "target5")
  
).toDF("profile_id", "name", "target")

display(ref)

In [ ]:
dfTweets

.join(ref, dfTweets("user_id") === ref("profile_id"))

.createOrReplaceTempView("known_users")

In [ ]:
/** ml **/

In [ ]:
val finalColumns = Seq("text", "timestamp", "creation_time", "hashtags", "location")
val dfLive = dfTweets.filter(size($"hashtags") <= 3)

In [ ]:
import org.apache.spark.ml.{Pipeline, PipelineModel}

In [ ]:
val model: PipelineModel = PipelineModel.read.load(s"$S3_DIR/models/trump")

In [ ]:
val dfPrediction: DataFrame = model.transform(dfLive) 

In [ ]:
dfPrediction.printSchema

```
root
 |-- topic: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- is_retweet: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- creation_time: timestamp (nullable = true)
 |-- user_id: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- screen: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- token_raw: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- tf: vector (nullable = true)
 |-- idf: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)
```

In [ ]:
display(dfPrediction.select("prediction", finalColumns:_*))

![](https://raw.githubusercontent.com/DivLoic/mdd-structured-streaming/master/resources/final_result.png)

In [ ]:
/**
 * step 7: write to the sink
 */

![](https://raw.githubusercontent.com/DivLoic/mdd-structured-streaming/master/resources/sink.png)

In [ ]:
import scala.concurrent.duration._
import org.apache.spark.sql.streaming.{OutputMode, ProcessingTime, StreamingQuery}

`%fs ls /mnt/moisdeladata/data/tweets/prediction/`

In [ ]:
dfPrediction.select("prediction", finalColumns:_*)

  .writeStream.format("parquet")
  
  .option("path", s"$S3_DIR/data/tweets/prediction/table/")
  
  .option("checkpointLocation", s"$S3_DIR/checkpoints/prediction/")
  
  .trigger(ProcessingTime(0.5 seconds))
  
  .start()

`%fs ls /mnt/moisdeladata/data/tweets/prediction/`
```
    - part-00001-16dbff53-e686-4a57-a18a-26ca706034ad.snappy.parquet
    - part-00001-2b8afcb1-37ab-43c2-8026-b5ad03dfe22f.snappy.parquet
    - part-00001-3474a0e2-6e56-4073-9fe6-790c4b5c65f6.snappy.parquet
    - part-00001-686a7735-41a4-43c7-a1af-a86461d2b75f.snappy.parquet
    - ...
```    

In [ ]:
dbutils.fs.ls("/mnt/moisdeladata/data/tweets/prediction/table/").size

Sources: 
- *[Apache Spark](http://spark.apache.org) documentation*

In [ ]:
println(
    """
    | __  __ _____ ____   ____ ___
    ||  \/  | ____|  _ \ / ___|_ _|
    || |\/| |  _| | |_) | |    | | 
    || |  | | |___|  _ <| |___ | | 
    ||_|  |_|_____|_| \_\\____|___|
    |
    """
)